In [1]:
from ccxt import binance,okex5,bitget,bybit,binanceusdm
import pandas as pd
from random import random,randint,uniform
from datetime import datetime

In [2]:
#binance
binance_ex = binanceusdm_ex = binanceusdm({
    "option":{
        "defaultType":"swap"
    }
})
binance_ex.load_markets()
random_open_order_symbols = [binance_ex.symbols[i] for i in sorted([int(random()*len(binance_ex.symbols)) for i in range(25)])]

['1INCH/USDT:USDT',
 'BTCDOM/USDT:USDT',
 'CHZ/USDT:USDT',
 'ENS/USDT:USDT',
 'FTT/BUSD:BUSD',
 'FTT/BUSD:BUSD',
 'GAL/USDT:USDT',
 'GRT/USDT:USDT',
 'IOST/USDT:USDT',
 'LIT/USDT:USDT',
 'LRC/USDT:USDT',
 'MATIC/USDT:USDT',
 'OMG/USDT:USDT',
 'QNT/USDT:USDT',
 'QTUM/USDT:USDT',
 'SSV/USDT:USDT',
 'THETA/USDT:USDT',
 'TLM/BUSD:BUSD',
 'TOMO/USDT:USDT',
 'USDC/USDT:USDT',
 'WAVES/USDT:USDT',
 'XRP/BUSD:BUSD',
 'YFI/USDT:USDT',
 'ZIL/USDT:USDT',
 'ZRX/USDT:USDT']

In [3]:
entry_data = {}
for symbol in random_open_order_symbols:
    side = randint(0,1)
    symbol_history_df = pd.DataFrame(binance_ex.fetch_ohlcv(symbol, '15m', limit=1000))
    symbol_max_value, symbol_min_value = binance_ex.markets[symbol]['limits']['price']['max'], binance_ex.markets[symbol]['limits']['price']['min']
    quantity = uniform(symbol_min_value,symbol_max_value)
    if quantity*symbol_history_df[4][1] > 1000:
        quantity = round(1000/symbol_history_df[4][1],4)
    if side == 0:
        most_lowest_price_index = symbol_history_df[4].idxmin()
        entry_data[symbol] = {
            'side': 'buy',
            'entry_price': symbol_history_df[4][most_lowest_price_index],
            'entry_time': datetime.fromtimestamp(symbol_history_df[0][most_lowest_price_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
            'entry_quantity': quantity,
            'entry_leverage': randint(1,25),
            'entry_type': 'market',
            'entry_class_SF': 'future',
            'exchange': 'binance',
        }
    else:
        most_highest_price_index = symbol_history_df[4].idxmax()
        entry_data[symbol] = {
            'side': 'sell',
            'entry_price': symbol_history_df[4][most_highest_price_index],
            'entry_time': datetime.fromtimestamp(symbol_history_df[0][most_highest_price_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
            'entry_quantity': quantity,
            'entry_leverage': randint(1,25),
            'entry_type': 'market',
            'entry_class_SF': 'future',
            'exchange': 'binance',
        }

# entry_data #fake data , regenerate it before use
    

In [4]:
def Caculate_Unrealized_Pnl(open_orders_data:dict):
    for symbol in open_orders_data:
        symbol_history_df = pd.DataFrame(binance_ex.fetch_ohlcv(symbol, '1m', limit=10))
        most_recent_price_index = symbol_history_df[4].idxmax()
        most_recent_price = symbol_history_df[4][most_recent_price_index]
        if open_orders_data[symbol]['side'] == 'buy':
            open_orders_data[symbol]['Unrealized'] = round((most_recent_price - open_orders_data[symbol]['entry_price'])*open_orders_data[symbol]['entry_quantity'],2)
        else:
            open_orders_data[symbol]['Unrealized'] = round((open_orders_data[symbol]['entry_price'] - most_recent_price)*open_orders_data[symbol]['entry_quantity'],2)
    return open_orders_data

open_orders_data = Caculate_Unrealized_Pnl(entry_data)

In [5]:
#data to pandas data frame
open_orders_data = pd.DataFrame(open_orders_data).T
open_orders_data

,side,entry_price,entry_time,entry_quantity,entry_leverage,entry_type,entry_class_SF,exchange,Unrealized
1INCH/USDT:USDT,sell,0.576,2023-03-19 16:30:00,2246.1815,24,market,future,binance,154.54
BTCDOM/USDT:USDT,sell,1684.0,2023-03-21 07:15:00,0.6991,12,market,future,binance,9.51
CHZ/USDT:USDT,sell,0.12586,2023-03-18 16:45:00,59.983853,1,market,future,binance,0.54
ENS/USDT:USDT,buy,11.521,2023-03-12 00:00:00,81.057,1,market,future,binance,157.25
FTT/BUSD:BUSD,buy,1.678,2023-03-11 06:15:00,595.9476,16,market,future,binance,0.0
GAL/USDT:USDT,buy,1.3054,2023-03-12 00:15:00,706.115,6,market,future,binance,325.73
GRT/USDT:USDT,sell,0.17471,2023-03-18 20:45:00,24.937083,9,market,future,binance,0.73
IOST/USDT:USDT,sell,0.011519,2023-03-20 10:00:00,1134.161547,23,market,future,binance,1.32
LIT/USDT:USDT,buy,0.804,2023-03-11 23:45:00,1116.0714,19,market,future,binance,275.67
LRC/USDT:USDT,buy,0.2573,2023-03-12 00:15:00,69.142178,6,market,future,binance,5.43
